In [1]:
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [11]:
text = "my name is kartheek. my age is 19. i am studying in NIT MANIPUR."

In [12]:
doc = nlp(text)
for sent in doc.sents:
    print(sent)

my name is kartheek.. my age is 19.
i am studying in NIT MANIPUR.


In [13]:
for token in doc:
    print(token.text)

my
name
is
kartheek
..
my
age
is
19
.
i
am
studying
in
NIT
MANIPUR
.


### create_pipe()

In [ ]:

sent = nlp.create_pipe('sentencizer')
nlp.add_pipe(sent, before = 'parser')

In [15]:
doc = nlp(text)

In [16]:
for sent in doc.sents:
    print(sent)

my name is kartheek.. my age is 19.
i am studying in NIT MANIPUR.


### Stop_words

In [1]:
from spacy.lang.en.stop_words import STOP_WORDS

In [10]:
stopwords = list(STOP_WORDS)
stopwords

['’m',
 'even',
 'three',
 '‘s',
 'up',
 'move',
 'itself',
 'is',
 'why',
 'empty',
 '’ll',
 "'s",
 'almost',
 'both',
 'whence',
 'would',
 'down',
 'under',
 'while',
 'same',
 'for',
 'could',
 'might',
 'six',
 'then',
 'many',
 'my',
 'give',
 'due',
 'namely',
 'whole',
 'ever',
 '’s',
 'whatever',
 'everything',
 'throughout',
 'what',
 'forty',
 'or',
 'us',
 'does',
 'none',
 'least',
 'i',
 'hundred',
 'after',
 'have',
 "'m",
 'her',
 'become',
 'other',
 'its',
 'side',
 'n’t',
 'rather',
 're',
 'became',
 'your',
 'twelve',
 'when',
 "'d",
 'anything',
 'thru',
 'everyone',
 'please',
 'off',
 'anyone',
 'call',
 'of',
 "'ll",
 'upon',
 'whereby',
 'me',
 'she',
 'nor',
 'during',
 'hers',
 'used',
 'against',
 'once',
 'where',
 'the',
 'behind',
 'nobody',
 'mostly',
 'less',
 'most',
 'hereupon',
 'over',
 'therefore',
 'which',
 'still',
 'twenty',
 'that',
 'him',
 'anyway',
 'toward',
 'done',
 'herself',
 'must',
 'name',
 'was',
 'do',
 'being',
 'beforehand',
 '

In [11]:
len(stopwords)

326

# sentimet classification on IMDB and Amazon datasets

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [1]:
import pandas as pd

In [2]:
data_yelp = pd.read_csv('datasets/yelp_labelled.txt', sep = '\t', header = None)    
#header = None --> otherwise 1st row will be header....

In [3]:
data_yelp

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [4]:
column_name = ['Review', 'Sentiment']
data_yelp.columns = column_name

In [5]:
data_yelp

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [6]:
data_amazon = pd.read_csv('datasets/amazon_cells_labelled.txt', sep='\t', header = None)
data_amazon.columns = column_name

In [7]:
data_amazon

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [8]:
data_imdb= pd.read_csv('datasets/imdb_labelled.txt', sep='\t', header = None)
data_imdb.columns = column_name

In [9]:
data_imdb

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [10]:
data = data_yelp.append([data_amazon, data_imdb], ignore_index = True)
data

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2743,I just got bored watching Jessice Lange take h...,0
2744,"Unfortunately, any virtue in this film's produ...",0
2745,"In a word, it is embarrassing.",0
2746,Exceptionally bad!,0


## saving dataset

In [12]:
data.to_csv('datasets/saved IMDB dataset.csv', index= False)

#### data processing

In [11]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [12]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

#### Tokenization

In [1]:
import string  #it's a python package.
import spacy

nlp = spacy.load('en_core_web_sm')

In [2]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
len(stopwords)

326

In [18]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens =[]
    for token in doc:
        if token.lemma_ !='-PRON-':
            temp = token.lemma_.lower().strip()  #since all our data prepocessing steps are in lower format..
            
        else:
            temp = token.lower_
        tokens.append(temp)
        
    #removing stopwords and punctuations
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens
        

In [19]:
text_data_cleaning("Hello how are you. Like this videos")

['hello', 'like', 'video']

### vectorization feature engineering (TF-IDF)

In [24]:
from sklearn.svm import LinearSVC

In [25]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [26]:
x = data['Review']
y = data['Sentiment']

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state = 42)

In [29]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
#'tfidf' -- is a name of tfidf object similarly 'clf' -- is a name of classifier.

In [30]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x00000174A2E86EE0>)),
                ('clf', LinearSVC())])

In [31]:
y_pred = clf.predict(x_test)  #so here pipeline is used by not transforming x_test again as it automatically transform data..

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       285
           1       0.79      0.74      0.77       265

    accuracy                           0.78       550
   macro avg       0.78      0.78      0.78       550
weighted avg       0.78      0.78      0.78       550



In [33]:
confusion_matrix(y_test, y_pred)

array([[233,  52],
       [ 68, 197]], dtype=int64)

#### TESTING TIME

In [34]:
clf.predict(['wow, this is amazing lession'])   #positive sentement

array([1], dtype=int64)

In [35]:
clf.predict(['wow, this sucks'])  #negative sentement

array([0], dtype=int64)